In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox, Layout, Box
from IPython.display import clear_output
import _pickle as pickle
from w2v_model import word2vec
import get_answer

In [3]:
ob = word2vec()
path = "../data_prep/data/"
res = get_answer.get_dir_list(path)
print (res)
get_answer.ob = ob
driver = get_answer.Driver()
fp = open(path+"./../PR.txt", 'w')
for cat in res:
    print ("\n\n\n\n==================================\n\n\n\n")
    print ("                 "+cat+"               ")
    print ("\n\n\n\n==================================\n\n\n\n")
    fp.write(cat)
    fp.write("\n")
    driver.run_w2v(path+cat, fp)
fp.close()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/shashwat/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /home/shashwat/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /home/shashwat/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/shashwat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['Musical_Instruments']








                 Musical_Instruments               








loading data
reading dict
reading word vec
reading model
done


In [4]:
result= None
def ask_question(b):
    global result
    global category, product, question_text    
    result = get_answer.review_2_sent(question_text.value, 4, product.value)

In [5]:
categories = ["Select...", "Musical Instruments"]
# "Arts Crafts and Sewing", "Automotive", "Baby", "Beauty", \
# "Cell Phones and Accessories", "Clothing Shoes and Jewelry", "Electronics", \
# "Grocery and Gourmet Food", "Health and Personal Care", "Home and Kitchen", \
# "Industrial and Scientific", , "Office Products", \
# "Patio Lawn and Garden", "Pet Supplies", "Software", "Sports and Outdoors"
products = ["P1", "B00005ML71", "B000VTPR08", "B0000AQRSR","P4", "P5"]
# with open("./cs670project/data_prep/data/Musical_Instruments/Musical_Instruments_QA.json", 'rb') as f:
#     data = f.read()
#     print (type(f))
#     data = pickle.load(data[1:-1])

# print (data[1:-1])

In [6]:
category = widgets.Dropdown(
    options=categories,
    value='Select...',
    description='Category:',
    disabled=False,
)

product = widgets.Dropdown(
    options=products,
    value='P1',
    description='Product:',
    disabled=False,
)

question_text = widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='Question:',
    disabled=False
)

submit = widgets.Button(
    description='Ask me!',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)
submit.on_click(ask_question)

box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    border='none',
                    width='60%')

items = [category, product, question_text, submit]
box =Box(children=items, layout=box_layout)

In [7]:
box

Box(children=(Dropdown(description='Category:', options=('Select...', 'Musical Instruments'), value='Select...…

Answer1:  What s also great is you save some money as an XLR cable is included.I like this mic ok but the SM58 is worth the extra money I think  especially if you expect to push its capability.It gives a clean crisp reproduction of every flaw I have.I m very happy with this purchase.Shure makes excellent stuff.The Gain on this mic isn t the greatest  but for speeches and soft recordings  this guy can handle it all very well. 



In [14]:
result

{'negative': '',
 'positive': 'I haven t heard a hint of feedback and the on off switch is a plus.It s perfect for vocals  podcasting  video recording and so forth.I have a regular gig at a medium sized club that is basically one large room.I would have no hesitation using this for bigger rooms with a full band and PA.I m very happy with this purchase.What s also great is you save some money as an XLR cable is included.',
 'reviews': ["I use this for Video Exchanges on a guitar instruction website. It's perfect for vocals, podcasting, video recording and so forth. Shure makes excellent stuff. What's also great is you save some money as an XLR cable is included. I'm very happy with this purchase.",
  "The Gain on this mic isn't the greatest, but for speeches and soft recordings: this guy can handle it all very well!",
  'I have a few Shure SM58s which cost more.  I like the open sound of the PG48 but vocals seem to break up more at higher volumes than they do with the SM58s.  I like thi